# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 7 2022 10:20AM,248156,12,8622352,HealthCaps LLC,Released
1,Oct 7 2022 10:14AM,248154,10,8619279,Eywa Pharma Inc.,Released
2,Oct 7 2022 10:14AM,248150,15,VT80178974,"Virtus Pharmaceuticals, LLC",Released
3,Oct 7 2022 10:14AM,248150,15,VT80178975,"Virtus Pharmaceuticals, LLC",Released
4,Oct 7 2022 10:14AM,248150,15,VT80178976,"Virtus Pharmaceuticals, LLC",Released
5,Oct 7 2022 10:14AM,248150,15,VT80178977,"Virtus Pharmaceuticals, LLC",Released
6,Oct 7 2022 10:14AM,248155,20,8622350,"Adare Pharmaceuticals, Inc.",Executing
7,Oct 7 2022 10:13AM,248152,10,8618106,Eywa Pharma Inc.,Released
8,Oct 7 2022 10:13AM,248152,10,8619293,Eywa Pharma Inc.,Released
9,Oct 7 2022 10:13AM,248152,10,8619294,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,248151,Released,1
32,248152,Released,5
33,248154,Released,1
34,248155,Executing,1
35,248156,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248151,NaN,NaN,1.0
248152,NaN,NaN,5.0
248154,NaN,NaN,1.0
248155,NaN,1.0,NaN
248156,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248074,0.0,1.0,41.0
248085,0.0,0.0,1.0
248097,0.0,1.0,0.0
248115,0.0,1.0,8.0
248116,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248074,0,1,41
248085,0,0,1
248097,0,1,0
248115,0,1,8
248116,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248074,0,1,41
1,248085,0,0,1
2,248097,0,1,0
3,248115,0,1,8
4,248116,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248074,,1,41
1,248085,,,1
2,248097,,1,
3,248115,,1,8
4,248116,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 7 2022 10:20AM,248156,12,HealthCaps LLC
1,Oct 7 2022 10:14AM,248154,10,Eywa Pharma Inc.
2,Oct 7 2022 10:14AM,248150,15,"Virtus Pharmaceuticals, LLC"
6,Oct 7 2022 10:14AM,248155,20,"Adare Pharmaceuticals, Inc."
7,Oct 7 2022 10:13AM,248152,10,Eywa Pharma Inc.
12,Oct 7 2022 10:13AM,248149,15,"Person & Covey, Inc."
37,Oct 7 2022 10:10AM,248151,10,Methapharm-G
38,Oct 7 2022 9:58AM,248148,15,"Alliance Pharma, Inc."
49,Oct 7 2022 9:30AM,248145,10,"Citieffe, Inc."
50,Oct 7 2022 9:25AM,248141,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 7 2022 10:20AM,248156,12,HealthCaps LLC,,,1
1,Oct 7 2022 10:14AM,248154,10,Eywa Pharma Inc.,,,1
2,Oct 7 2022 10:14AM,248150,15,"Virtus Pharmaceuticals, LLC",,,4
3,Oct 7 2022 10:14AM,248155,20,"Adare Pharmaceuticals, Inc.",,1,
4,Oct 7 2022 10:13AM,248152,10,Eywa Pharma Inc.,,,5
5,Oct 7 2022 10:13AM,248149,15,"Person & Covey, Inc.",,3,22
6,Oct 7 2022 10:10AM,248151,10,Methapharm-G,,,1
7,Oct 7 2022 9:58AM,248148,15,"Alliance Pharma, Inc.",,2,9
8,Oct 7 2022 9:30AM,248145,10,"Citieffe, Inc.",,,1
9,Oct 7 2022 9:25AM,248141,10,Bio-PRF,,,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 10:20AM,248156,12,HealthCaps LLC,1,,
1,Oct 7 2022 10:14AM,248154,10,Eywa Pharma Inc.,1,,
2,Oct 7 2022 10:14AM,248150,15,"Virtus Pharmaceuticals, LLC",4,,
3,Oct 7 2022 10:14AM,248155,20,"Adare Pharmaceuticals, Inc.",,1,
4,Oct 7 2022 10:13AM,248152,10,Eywa Pharma Inc.,5,,
5,Oct 7 2022 10:13AM,248149,15,"Person & Covey, Inc.",22,3,
6,Oct 7 2022 10:10AM,248151,10,Methapharm-G,1,,
7,Oct 7 2022 9:58AM,248148,15,"Alliance Pharma, Inc.",9,2,
8,Oct 7 2022 9:30AM,248145,10,"Citieffe, Inc.",1,,
9,Oct 7 2022 9:25AM,248141,10,Bio-PRF,8,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 10:20AM,248156,12,HealthCaps LLC,1,,
1,Oct 7 2022 10:14AM,248154,10,Eywa Pharma Inc.,1,,
2,Oct 7 2022 10:14AM,248150,15,"Virtus Pharmaceuticals, LLC",4,,
3,Oct 7 2022 10:14AM,248155,20,"Adare Pharmaceuticals, Inc.",,1,
4,Oct 7 2022 10:13AM,248152,10,Eywa Pharma Inc.,5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 10:20AM,248156,12,HealthCaps LLC,1.0,NaN,NaN
1,Oct 7 2022 10:14AM,248154,10,Eywa Pharma Inc.,1.0,NaN,NaN
2,Oct 7 2022 10:14AM,248150,15,"Virtus Pharmaceuticals, LLC",4.0,NaN,NaN
3,Oct 7 2022 10:14AM,248155,20,"Adare Pharmaceuticals, Inc.",NaN,1.0,NaN
4,Oct 7 2022 10:13AM,248152,10,Eywa Pharma Inc.,5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 7 2022 10:20AM,248156,12,HealthCaps LLC,1.0,0.0,0.0
1,Oct 7 2022 10:14AM,248154,10,Eywa Pharma Inc.,1.0,0.0,0.0
2,Oct 7 2022 10:14AM,248150,15,"Virtus Pharmaceuticals, LLC",4.0,0.0,0.0
3,Oct 7 2022 10:14AM,248155,20,"Adare Pharmaceuticals, Inc.",0.0,1.0,0.0
4,Oct 7 2022 10:13AM,248152,10,Eywa Pharma Inc.,5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1985136,52.0,44.0,1.0
12,248156,1.0,0.0,0.0
15,992521,76.0,6.0,0.0
16,2977444,44.0,4.0,0.0
20,496240,1.0,1.0,0.0
21,496253,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1985136,52.0,44.0,1.0
1,12,248156,1.0,0.0,0.0
2,15,992521,76.0,6.0,0.0
3,16,2977444,44.0,4.0,0.0
4,20,496240,1.0,1.0,0.0
5,21,496253,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,52.0,44.0,1.0
1,12,1.0,0.0,0.0
2,15,76.0,6.0,0.0
3,16,44.0,4.0,0.0
4,20,1.0,1.0,0.0
5,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,52.0
1,12,Released,1.0
2,15,Released,76.0
3,16,Released,44.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,20,21
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0
Executing,44.0,0.0,6.0,4.0,1.0,1.0
Released,52.0,1.0,76.0,44.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,44.0,0.0,6.0,4.0,1.0,1.0
2,Released,52.0,1.0,76.0,44.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,44.0,0.0,6.0,4.0,1.0,1.0
2,Released,52.0,1.0,76.0,44.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()